In [47]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [48]:
# Rutas de los datasets originales
ruta_credits = r'C:\HENRY\PROYECTOS HENRY 11\Movies\credits1.json'
#ruta_credits_csv = r'C:\HENRY\PROYECTOS HENRY 11\Movies\movies_dataset.csv'
ruta_movies = r'C:\HENRY\PROYECTOS HENRY 11\Movies\movies2.json'
ruta_credits_reduce = r'C:\HENRY\PROYECTOS HENRY 11\Primer Proyecto Individual\Datasets\newCredits.pkl'



In [49]:
credits_lista = []
with open(ruta_credits, 'rt', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                credits_lista.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error en la línea {line_num}: {line}")
                print(f"Detalle del error: {e}")
df_credits = pd.DataFrame(credits_lista)


In [50]:
df_credits.head()

,Cast,Crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ''Savannah 'Vanna...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [51]:
#df = pd.read_csv(ruta_credits_csv)
#df.to_json('movies2.json', orient='records', lines=True)

In [52]:
movies_lista = []
with open(ruta_movies, 'rt', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        line = line.strip()
        if line:
            try:
                movies_lista.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error en la línea {line_num}: {line}")
                print(f"Detalle del error: {e}")
df_movies = pd.DataFrame(movies_lista)


In [53]:
df_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415


In [54]:
#Cambiar valores nulos en revenue por 0
df_movies['revenue'] = np.where(pd.isnull(df_movies['revenue']), 0, df_movies['revenue'])

In [55]:
#Cambiar valores nulos en revenue por 0
df_movies['budget'] = np.where(pd.isnull(df_movies['budget']), 0, df_movies['budget'])

In [56]:
df_movies['release_date'].dropna()

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45457    1991-05-13
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45379, dtype: object

In [57]:
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

DESEMPAQUETAR

In [58]:
# Desanidar la columna 'Cast'
df_credits_expanded_cast = df_credits.explode('Cast')

# Normalizar la columna 'Cast'
df_credits_cast = pd.json_normalize(df_credits_expanded_cast['Cast'])

# Renombrar las columnas de 'Cast'
df_credits_cast.columns = [f"cast_{col}" for col in df_credits_cast.columns]

# Combinar los DataFrames con 'id' y 'Crew'
df_credits_cast = pd.concat([
    df_credits_expanded_cast[['id']].reset_index(drop=True),
    df_credits_cast.reset_index(drop=True)
], axis=1)

print(df_credits_cast.head(20))


      id  cast_cast_id                  cast_character  \
0    862          14.0                   Woody (voice)   
1    862          15.0          Buzz Lightyear (voice)   
2    862          16.0         Mr. Potato Head (voice)   
3    862          17.0              Slinky Dog (voice)   
4    862          18.0                     Rex (voice)   
5    862          19.0                    Hamm (voice)   
6    862          20.0                 Bo Peep (voice)   
7    862          26.0                    Andy (voice)   
8    862          22.0                     Sid (voice)   
9    862          23.0              Mrs. Davis (voice)   
10   862          24.0                Sergeant (voice)   
11   862          25.0                  Hannah (voice)   
12   862          27.0            TV Announcer (voice)   
13  8844           1.0                    Alan Parrish   
14  8844           8.0  Samuel Alan Parrish / Van Pelt   
15  8844           2.0                    Judy Sheperd   
16  8844      

In [59]:
#df_credits_exp.to_json('df_combined.json', lines=True, orient='records')
#df_credits_exp.to_parquet('df_credits_exp.parquet',engine='pyarrow' )

In [60]:
# Cambiamos el tipo de dato de la columna 'release_date' str a datetime
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

# Verificamos el tipo de dato de la columna
print(type(df_movies['release_date'].loc[2]))


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [61]:
#Creamos la columna 'release_year', la cual se obtiene de sacar el año de la columna 'release_date'
df_movies['release_year']=df_movies['release_date'].dt.year

In [62]:
#Creamos columna 'return', la cual es el resultado de revenue / budget
df_movies['return']=df_movies['revenue']/df_movies['budget']

In [63]:
#Prueba
print(df_movies['revenue'].loc[1])
print(df_movies['budget'].loc[1])
print(df_movies['return'].loc[1])

262797249
65000000
4.0430346


In [64]:
#Eliminamos algunas columnas
df_movies = df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [65]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

In [66]:
#Transformando DataFrame a parquet 

#df_credits.to_parquet('credits.parquet', engine='pyarrow')


In [67]:
#df_movies.to_json('movies.json')

In [68]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
        Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [69]:
df_movies['original_language'].unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', None, 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay',
       'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu',
       'lb', 'si'], dtype=object)

In [70]:

def cantidad_filmaciones_mes1( Mes ):
    meses = {
        1: "enero", 2: "febrero", 3: "marzo", 4: "abril", 5: "mayo",
        6: "junio", 7: "julio", 8: "agosto", 9: "septiembre", 
        10: "octubre", 11: "noviembre", 12: "diciembre"
    }
    peliculas_espanol = df_movies[df_movies['original_language'] == 'es']
    peliculas_mes = peliculas_espanol[peliculas_espanol['release_date'].dt.month == Mes]
    print(f"{len(peliculas_mes)} películas fueron estrenadas en el mes de {meses[Mes]}")



In [71]:
cantidad_filmaciones_mes1(4)

97 películas fueron estrenadas en el mes de abril


In [72]:
#Función que calcula la cantidad de peliculas estrenadas en un mes determinado
def cantidad_filmaciones_mes( Mes ):
    Mes= Mes.lower() #Convierte el valor ingresado a minúsculas
    #asignamos un valor a cada mes en español que entre como parámetro
    meses = {
        "enero": 1, "febrero": 2, "marzo": 3, "abril":4, "mayo": 5,
        "junio":6, "julio":7, "agosto":8, "septiembre":9, 
         "octubre":10, "noviembre":11, "diciembre":12
    }
    if(Mes not in meses):#Si el mes seleccionado no está entre las opciones
        print("Ingrese un mes en español")
        return
    peliculas_mes = (df_movies[df_movies['release_date'].dt.month==meses[Mes]])
    print(f"{len(peliculas_mes)} películas fueron estrenadas en el mes de {Mes}")

In [73]:
cantidad_filmaciones_mes('ENERO')

5913 películas fueron estrenadas en el mes de enero


In [74]:
print(len(df_movies[df_movies['release_date'].dt.month==1]))

5913


def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
            Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [75]:
#Función que calcula la cantidad de peliculas estrenadas en un día determinado
def cantidad_filmaciones_dia( Dia ):
    Dia= Dia.lower() #Convierte el valor ingresado a minúsculas#Convierte el valor ingresado a minúsculas
    #asignamos un valor a cada día en español que entre como parámetro
    Dias = {
        "lunes": "Monday", "martes": "Tuesday", "miercoles": "Wednesday", "jueves":"Thursday", "viernes": "Friday",
        "sabado":"Saturday", "domingo":"Sunday"
    }
    if(Dia not in Dias):#Si el día seleccionado no está entre las opciones
        print("Ingrese un día en español")
        return
    peliculas_dia = (df_movies[df_movies['release_date'].dt.day_name()==Dias[Dia]])
    print(f"{len(peliculas_dia)} películas fueron estrenadas en el mes de {Dia}")

In [76]:
#Probando la función
cantidad_filmaciones_dia("lunes")

3503 películas fueron estrenadas en el mes de lunes


In [77]:
#Prueba
len(df_movies[df_movies['release_date'].dt.day_name()=='Monday'])

3503

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.

In [78]:
def score_titulo(titulo_de_la_filmación):
    titulo_de_la_filmación = titulo_de_la_filmación.lower()
    # Convertir los títulos del DataFrame a minúsculas para la comparación
    df_movies['title_lower'] = df_movies['title'].str.lower()
    # Filtrar las filas donde el título coincida con el título de la filmación
    coincidencias = df_movies[df_movies['title_lower'] == titulo_de_la_filmación]

    # Si hay coincidencias, devuelve el título, el año de lanzamiento, y la puntuación
    if not coincidencias.empty:
        titulo = coincidencias['title'].values[0]
        año = int(coincidencias['release_year'].values[0])
        puntuacion = float(coincidencias['vote_average'].values[0] / coincidencias['popularity'].values[0])
        # Aplicamos la función round para redondear solo a 2 decimales
        puntuacion = round(puntuacion, 2)
        #return titulo, año, puntuacion
        return print(f" La película {titulo} fue estrenada en el año {año} con un score/popularidad de {puntuacion} ")
    else:
        return "No se encontró la filmación", None, None

# Ejemplo de uso
resultado = score_titulo("Sense and Sensibility")
print(resultado)


 La película Sense and Sensibility fue estrenada en el año 1995 con un score/popularidad de 0.67 
None


In [79]:
score_titulo("Casino")

 La película Casino fue estrenada en el año 1995 con un score/popularidad de 0.77 


In [80]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'title_lower'],
      dtype='object')

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [81]:
def votos_titulo(titulo_de_la_filmación):
    titulo_de_la_filmación = titulo_de_la_filmación.lower()
    # Convertir los títulos del DataFrame a minúsculas para la comparación
    df_movies['title_lower'] = df_movies['title'].str.lower()
    # Filtrar las filas donde el título coincida con el título de la filmación
    coincidencias = df_movies[df_movies['title_lower'] == titulo_de_la_filmación]
    
    if not coincidencias.empty:
        votos = coincidencias.iloc[0]['vote_count']
        promedio = coincidencias.iloc[0]['vote_average']
        if votos >= 2000:
            return f"La película {coincidencias.iloc[0]['title']} fue estrenada en el año {int(coincidencias.iloc[0]['release_year'])}. La misma cuenta con un total de {votos} valoraciones, con un promedio de {promedio}"
    return "No se encontraron coincidencias o la película no tiene suficientes valoraciones."


In [82]:
votos_titulo('Se7en')

'La película Se7en fue estrenada en el año 1995. La misma cuenta con un total de 5915 valoraciones, con un promedio de 8.1'

In [83]:
df_credits.columns

Index(['Cast', 'Crew', 'id'], dtype='object')

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [84]:
print(df_movies.columns)


Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'title_lower'],
      dtype='object')


In [85]:
def get_actor(nombre_actor):
    nombre_actor = nombre_actor.lower()
    
    # Convertir los nombres del DataFrame a minúsculas para la comparación
    df_credits_cast['cast_name'] = df_credits_cast['cast_name'].str.lower()
    
    # Filtrar las filas donde el nombre coincida con el nombre_actor
    coincidencias = df_credits_cast[df_credits_cast['cast_name'] == nombre_actor]
    
    # Verificar que haya coincidencias
    if coincidencias.empty:
        return nombre_actor, 0, 0.0, 0.0
        
    # Obtener los IDs únicos y contar la cantidad de películas
    cant_films = len(coincidencias['id'].unique())
    
    # Verificar que la columna 'return' exista
    if 'return' not in df_movies.columns:
        return nombre_actor, cant_films, 0.0, 0.0
    
    # Seleccionar las películas del actor
    actor_movies = df_movies[df_movies['id'].isin(coincidencias['id'])]
    
    # Asegurarse de que 'return' no contiene valores infinitos o nulos
    actor_movies = actor_movies.replace([np.inf, -np.inf], np.nan).dropna(subset=['return'])
    
      
    # Calcular el retorno total y el promedio de retorno
    total_return = float(round(actor_movies['return'].sum(), 2))
    avg_return = float(round(actor_movies['popularity'].sum() / cant_films if cant_films > 0 else 0.0, 2))
    
    return f"El actor {nombre_actor} ha participado de {cant_films} cantidad de filmaciones, el mismo ha conseguido un retorno de {total_return} con un promedio de {avg_return} por filmación"
    
# Prueba la función
print(get_actor('Johnny Depp'))
print(get_actor('Alfred Molina'))


El actor johnny depp ha participado de 69 cantidad de filmaciones, el mismo ha conseguido un retorno de 139.67 con un promedio de 11.21 por filmación
El actor alfred molina ha participado de 66 cantidad de filmaciones, el mismo ha conseguido un retorno de 71.81 con un promedio de 3.98 por filmación


In [86]:

def get_actor(nombre_actor: str):
    nombre_actor = nombre_actor.lower()
    
    # Filtrar las filas donde el trabajo sea "Actor" y el nombre sea el especificado
    df_actor = df_credits[df_credits['Cast'].apply(lambda cast: any(member['name'].lower() == nombre_actor for member in cast))]
    
    # Obtener los IDs únicos y contar la cantidad de películas
    cant_films = len(df_actor['id'].unique())
    
    # Filtrar las películas en df_movies cuyo 'id' está en df_actor
    peliculas = df_movies[df_movies['id'].isin(df_actor['id'])]
    
    # Manejar los valores NaN en la columna 'return' antes de sumar
    peliculas['return'].fillna(0, inplace=True)
    
    # Calcular el retorno total y el promedio de retorno por filmación
    total_return = float(round(peliculas['return'].sum(), 2))
    avg_return_per_film = total_return / cant_films if cant_films > 0 else 0.0
    
    return {
        "actor": nombre_actor,
        "cantidad_filmes": cant_films,
        "total_return": total_return,
        "avg_return": avg_return_per_film
    }


print(get_actor('Johnny Depp'))


{'actor': 'johnny depp', 'cantidad_filmes': 69, 'total_return': inf, 'avg_return': inf}


C:\Users\diani\AppData\Local\Temp\ipykernel_26704\3297257240.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  peliculas['return'].fillna(0, inplace=True)
C:\Users\diani\AppData\Local\Temp\ipykernel_26704\3297257240.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peliculas['return'].fillna(0, inplace=True)


In [87]:
# Filtrar las filas donde el nombre del cast sea 'Johnny Depp'
conteo = df_credits_cast[df_credits_cast['cast_name'] == 'johnny depp']['id'].unique()
print(conteo)



[  2086   1909    522   1587    922    792    377   4104   9366   1878
  11284    162  51679   2900   2668    622    392   5001   4133  29572
    768  21189     22   1428   1586  11044    866   9768   9956    118
  14653  17824  61675   3933   7548  53367  17700     58    285  25740
  13885  18893  13003  11322   8054  12155  37659  37710  44896   1865
  71880  23514  64688  62213  57201 200063 157353 140456 246403 214096
  32624 224141 166426 210860 259316 261023 290825 241259 382079]


def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [88]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'title_lower'],
      dtype='object')

In [89]:
def get_director( nombre_director ):
   
    # Filtrar las filas donde el trabajo sea "Director" y el nombre sea "nombre_director"
    df_directors = df_credits[df_credits['Crew'].apply(lambda crew: any(member['job'] == 'Director' and member['name'] == nombre_director for member in crew))]

    # Extraer los nombres de los directores
    directors = df_directors['Crew'].apply(lambda crew: [member['name'] for member in crew if member['job'] == 'Director' and member['name'] == nombre_director])

    # Añadir los nombres de los directores al DataFrame original
    df_directors['director_names'] = directors

    #comparación peliculas
    peliculas = df_movies[df_movies['id'].isin(df_directors['id'])]

    # Manejar los valores NaN en la columna 'return' antes de sumar
    peliculas['return'].fillna(0, inplace=True)
   
    print(f"El director {nombre_director} tiene un nivel de retorno de {round(sum(peliculas['return']),2)}")
    print(peliculas[['id','title', 'release_year','revenue', 'budget','return']])

get_director('Gary Trousdale')

El director Gary Trousdale tiene un nivel de retorno de 17.65
          id                                         title  release_year  \
588    10020                          Beauty and the Beast        1991.0   
763    10545                   The Hunchback of Notre Dame        1996.0   
4238   10865                     Atlantis: The Lost Empire        2001.0   
13152  13394                               Shrek the Halls        2007.0   
16099  48466                              Scared Shrekless        2010.0   
25025  10258  The Madagascar Penguins in a Christmas Caper        2005.0   

         revenue     budget     return  
588    377350553   25000000  15.094022  
763    100138851  100000000   1.001389  
4238   186053725  120000000   1.550448  
13152          0   60000000   0.000000  
16099          0          0   0.000000  
25025          0          0   0.000000  


C:\Users\diani\AppData\Local\Temp\ipykernel_26704\2670479068.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_directors['director_names'] = directors
C:\Users\diani\AppData\Local\Temp\ipykernel_26704\2670479068.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  peliculas['return

In [91]:
df_credits_pkl = pd.read_pickle(r'C:\HENRY\PROYECTOS HENRY 11\Primer Proyecto Individual\Datasets\newCredits.pkl')
df_credits_pkl.head(5)
df_credits_pkl['directors']=str(df_credits_pkl['directors'])
print(type(df_credits_pkl['directors'].loc[1]))

<class 'str'>


In [92]:
def get_director(nombre_director):
    nombre_director = nombre_director.lower()

    # Asegurarse de que los valores en la columna 'directors' son cadenas
    df_credits_pkl['directors'] = df_credits_pkl['directors'].apply(lambda x: ', '.join(x).lower() if isinstance(x, list) else x.lower())

    # Filtrar las filas donde el nombre del director coincide con nombre_director
    coincidencias = df_credits_pkl[df_credits_pkl['directors'].str.contains(nombre_director)]

    # Verificar que haya coincidencias
    if coincidencias.empty:
        return nombre_director, 0, 0.0, 0.0

    # Comparación de películas
    peliculas = df_movies[df_movies['id'].isin(coincidencias['id'])]

    # Manejar los valores NaN en la columna 'return' antes de sumar
    peliculas['return'].fillna(0, inplace=True)

    # Resetear el índice del DataFrame 'peliculas'
    peliculas.reset_index(drop=True, inplace=True)
    # Obtener el nivel de retorno
    nivel_retorno = round(sum(peliculas['return']), 2)

    return f"El director {nombre_director} tiene un nivel de retorno de {nivel_retorno}", peliculas[['title', 'release_year', 'revenue', 'budget', 'return']]


# Prueba la función
print(get_director('Gary Trousdale'))


('gary trousdale', 0, 0.0, 0.0)


In [93]:
# Filtrar las filas donde el trabajo sea "Director" y el nombre sea "Gary Trousdale"
df_directors = df_credits[df_credits['Crew'].apply(lambda crew: any(member['job'] == 'Director' and member['name'] == 'Gary Trousdale' for member in crew))]

# Extraer los nombres de los directores
directors = df_directors['Crew'].apply(lambda crew: [member['name'] for member in crew if member['job'] == 'Director' and member['name'] == 'Gary Trousdale'])

# Añadir los nombres de los directores al DataFrame original
df_directors['director_names'] = directors

# Mostrar los IDs de las películas y los nombres de los directores
print(df_directors[['id', 'director_names']])



          id    director_names
588    10020  [Gary Trousdale]
763    10545  [Gary Trousdale]
4238   10865  [Gary Trousdale]
13152  13394  [Gary Trousdale]
16099  48466  [Gary Trousdale]
25025  10258  [Gary Trousdale]


C:\Users\diani\AppData\Local\Temp\ipykernel_26704\3560491521.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_directors['director_names'] = directors


Modelo: Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente.

In [94]:
# Similitud del coseno usando titulo

df_titulos = pd.DataFrame(df_movies, columns=["title"])
df_titulos.head(2)
vectorizer = TfidfVectorizer()
titulos_transform = vectorizer.fit_transform(df_titulos["title"])

def recomendacion( titulo, titulos_transform, df_titulos, top_n=5 ): 
# Encontrar el índice de la película seleccionada
    idx = df_titulos[df_titulos["title"] == titulo].index[0]
# Calcular la similitud del coseno entre la película seleccionada y el resto
    cosine_similarities = cosine_similarity(titulos_transform[idx], titulos_transform).flatten()

# Obtener los índices de las películas más similares
    similar_indices = cosine_similarities.argsort()[-top_n-1:-1][::-1]
# Recuperar los títulos de las películas similares
    similar_movies = df_movies.iloc[similar_indices]

    return similar_movies["title"].tolist()

movie_to_recommend = "Open Season"
recommendations = recomendacion(movie_to_recommend, titulos_transform, df_titulos)
print(f"Películas similares a '{movie_to_recommend}':\n", recommendations)


Películas similares a 'Open Season':
 ['Open Season', 'Open Season', 'Open Season 2', 'Open Season: Scared Silly', 'Open House']


In [95]:
# Similitud del coseno usando titulo y genero

def recommend_movies(movie_title, df= df_movies, title_col='title', genre_col='genres', num_recommendations=5):
    # Unir título y géneros en un texto
    def process_genres(genres):
        if isinstance(genres, str):  # Si es una cadena, intenta convertirla a lista de diccionarios
            try:
                genres = json.loads(genres.replace("'", "\""))
            except json.JSONDecodeError:
                return ""
        # Une los nombres de los géneros si es una lista de diccionarios
        return ' '.join([genre['name'] for genre in genres if isinstance(genre, dict) and 'name' in genre])

    # Aplicar la función de procesamiento a la columna 'genres'
    df['combined_features'] = df[title_col] + ' ' + df[genre_col].apply(process_genres)

    # Vectorizar las características combinadas
    vectorizer = TfidfVectorizer(stop_words='english')
    combined_matrix = vectorizer.fit_transform(df['combined_features'])

    # Encontrar el índice de la película objetivo
    target_index = df[df[title_col] == movie_title].index[0]

    # Calcular la similitud del coseno
    cosine_similarities = cosine_similarity(combined_matrix[target_index], combined_matrix).flatten()

    # Obtener índices de películas similares en orden descendente
    similar_indices = cosine_similarities.argsort()[::-1][1:num_recommendations + 1]

    # Crear un DataFrame con las recomendaciones y los puntajes de similitud
    recommendations = pd.DataFrame({
        'title': df.iloc[similar_indices][title_col].values,
        'similarity_score': cosine_similarities[similar_indices]
    }).sort_values(by='similarity_score', ascending=False)

    return recommendations


recomendaciones = recommend_movies('From Dusk Till Dawn')
print(recomendaciones)


                                           title  similarity_score
0       From Dusk Till Dawn 2: Texas Blood Money          0.789116
1  From Dusk Till Dawn 3: The Hangman's Daughter          0.739189
2                                           Dusk          0.615519
3                             Lights in the Dusk          0.491312
4                                   Another Dawn          0.454412


REDUCCIÓN DATA CREDITS: De Cast eliminamos "credit_id" y "profile_path" y de Crew conservamos unicamente el director de la pelicula. 

In [96]:
df_creditsP = pd.read_pickle(r'C:\HENRY\PROYECTOS HENRY 11\df_credits.pkl')

In [97]:
def get_directors(crew):   
    if isinstance(crew, list):
        return [member['name'] for member in crew if 'job' in member and member['job'] == 'Director' and 'name' in member]
    return []

# Crear el nuevo DataFrame
newCredits = pd.DataFrame({
    'id': df_creditsP['id'],
    'Cast': df_creditsP['Cast'],
    'directors': df_creditsP['Crew'].apply(get_directors)
})

# Mostrar las primeras filas del nuevo DataFrame
newCredits.head(3)


,id,Cast,directors
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)',...",[John Lasseter]
1,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",[Joe Johnston]
2,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",[Howard Deutch]


In [98]:
def remove_unwanted_keys(cast):
    keys_to_remove = ['profile_path', 'credit_id', 'cast_id', 'order']
    if isinstance(cast, list):
        return [{k: v for k, v in member.items() if k not in keys_to_remove} for member in cast]
    return []

newCredits['Cast'] = newCredits['Cast'].apply(remove_unwanted_keys)
newCredits.head(1)

,id,Cast,directors
0,862,"[{'character': 'Woody (voice)', 'gender': 2, '...",[John Lasseter]


In [99]:
#newCredits.to_pickle('newCredits.pkl')

In [100]:
len(newCredits)

45476

In [101]:
# Filtrar las filas donde 'return' es 0
vacios = df_movies[df_movies['return'] == 0]
print(len(vacios))
# Obtener los IDs de 'vacios' y 'newCredits'
ids_vacios = set(vacios['id'])
ids_newCredits = set(newCredits['id'])

# Encontrar las coincidencias
coincidencias = ids_vacios & ids_newCredits

# Eliminar las filas cuyos 'id' están en las coincidencias
newCredits = newCredits[~newCredits['id'].isin(coincidencias)]

print("IDs con coincidencias:", coincidencias)
print("Número de coincidencias:", len(coincidencias))
print(len(newCredits))
newCredits.columns



3509
IDs con coincidencias: {16387, 163845, 16391, 16392, 40969, 57353, 16395, 385031, 49160, 8209, 40979, 16403, 262165, 8216, 26, 24603, 40990, 139302, 98344, 65576, 24619, 122924, 24624, 122928, 122930, 294963, 16437, 262199, 57403, 139325, 114750, 98368, 98369, 68, 49220, 254023, 73800, 16455, 32845, 8274, 344147, 286805, 86, 32855, 114776, 91, 57436, 49250, 32872, 401513, 139374, 188529, 262260, 90231, 8316, 16515, 8326, 8332, 73869, 139406, 32911, 143, 123025, 180371, 148, 8341, 8342, 327833, 336029, 180383, 41123, 57510, 204968, 24757, 41142, 41141, 245950, 49347, 8388, 24775, 57544, 32969, 73935, 450768, 212, 90324, 216, 24794, 254172, 221, 32990, 286940, 8416, 73952, 229, 234, 16619, 73963, 114922, 385261, 57585, 147698, 278772, 24821, 247, 33016, 16633, 8440, 262391, 254200, 98557, 41214, 24831, 255, 16632, 139521, 65796, 24837, 262, 98567, 8457, 8460, 131343, 273, 336149, 98582, 418072, 352539, 418078, 24869, 8490, 16687, 221488, 309, 24885, 311, 57655, 315, 316, 318, 262474

Index(['id', 'Cast', 'directors'], dtype='object')

In [102]:
newCredits.to_pickle('newCredits1.pkl')